In [31]:
import pandas as pd
import numpy as np
from nltk.metrics import pk, windowdiff
from sklearn.metrics import f1_score

In [32]:
lstm = pd.read_json('results_4.json')
baseline = pd.read_json('logistic_results.json')

In [33]:
display(lstm.head())
display(baseline.head())

,real,prediction
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[0], [0], [0], [0], [1], [1], [1], [1], [1], ..."
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ..."


,real,prediction
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, ..."
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, ..."
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [34]:
lstm['prediction'] = lstm['prediction'].apply(lambda x: [item for sublist in x for item in sublist])

In [35]:
# Fraction of advetisement sentences
print(sum(lstm['real'].apply(sum)) / sum(lstm['real'].apply(len)))
print(sum(lstm['prediction'].apply(sum)) / sum(lstm['prediction'].apply(len)))
print(sum(baseline['prediction'].apply(sum)) / sum(baseline['prediction'].apply(len)))

0.756217629630308
0.7251191779128181
0.7761630436275879


In [36]:
def convert_to_boundaries(x):
    """
    Convert a list of binary values into a string of boundary markers.

    Parameters:
    x (list): A list of binary values (0s and 1s).

    Returns:
    str: A string where '1' indicates a boundary.
    """
    x = "".join(map(str, x))
    res = ""

    for i in range(len(x)):
        if i != 0:
            if x[i] != x[i-1]:
                res += '1'
            else:
                res += '0'
        else:
            res += '0'

    return res

In [37]:
lstm['true_boundaries'] = lstm['real'].apply(convert_to_boundaries)
lstm['pred_boundaries'] = lstm['prediction'].apply(convert_to_boundaries)

baseline['true_boundaries'] = baseline['real'].apply(convert_to_boundaries)
baseline['pred_boundaries'] = baseline['prediction'].apply(convert_to_boundaries)

In [38]:
def calculate_windowdiff(row):
    y_true = row['true_boundaries']
    y_pred = row['pred_boundaries']

    if y_true.count('1') != 0:
        k = int(round(len(y_true) / (y_true.count('1') * 2.0)))
    else:
        k = int(round(len(y_true) / 2.0))

    return windowdiff(y_true, y_pred, k)

In [39]:
def calculate_pk(row):
    y_true = row['true_boundaries']
    y_pred = row['pred_boundaries']

    if y_true.count('1') != 0:
        k = int(round(len(y_true) / (y_true.count('1') * 2.0)))
    else:
        k = int(round(len(y_true) / 2.0))

    return pk(y_true, y_pred, k)

In [40]:
lstm['windowdiff'] = lstm.apply(calculate_windowdiff, axis=1)
lstm['pk'] = lstm.apply(calculate_pk, axis=1)

baseline['windowdiff'] = baseline.apply(calculate_windowdiff, axis=1)
baseline['pk'] = baseline.apply(calculate_pk, axis=1)

In [41]:
def weighted_average(col):
    weighted_sum = (col * lstm['real'].apply(len)).sum()
    sum_of_weights = lstm['real'].apply(len).sum()

    res = weighted_sum / sum_of_weights
    return res

In [42]:
y_true_lstm = [item for sublist in lstm['real'] for item in sublist]
y_pred_lstm = [item for sublist in lstm['prediction'] for item in sublist]

In [43]:
y_true_baseline = [item for sublist in baseline['real'] for item in sublist]
y_pred_baseline = [item for sublist in baseline['prediction'] for item in sublist]

In [44]:
print(f"windowdiff: {weighted_average(lstm['windowdiff']) * 100:.2f}%")
print(f"pk: {weighted_average(lstm['pk']) * 100:.2f}%")
print(f"macro f1: {f1_score(y_true_lstm, y_pred_lstm, average='macro') * 100:.2f}%")
print(f"micro f1: {f1_score(y_true_lstm, y_pred_lstm, average='micro') * 100:.2f}%")

windowdiff: 15.59%
pk: 12.05%
macro f1: 86.15%
micro f1: 89.36%


In [45]:
print(f"windowdiff: {weighted_average(baseline['windowdiff']) * 100:.2f}%")
print(f"pk: {weighted_average(baseline['pk']) * 100:.2f}%")
print(f"macro f1: {f1_score(y_true_baseline, y_pred_baseline, average='macro') * 100:.2f}%")
print(f"micro f1: {f1_score(y_true_baseline, y_pred_baseline, average='micro') * 100:.2f}%")

windowdiff: 99.14%
pk: 76.00%
macro f1: 56.61%
micro f1: 68.91%
